In [34]:
!pip install nemoguardrails langchain openai transformers


In [35]:
!pip install -U langchain-openai


In [36]:
import os

# Set OpenAI API Key as an environment variable
os.environ["OPENAI_API_KEY"] = "hello"

In [102]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain

In [ ]:
task_prompt = """
You are a Python code generation assistant.
Your task is to generate production-ready code that follows industry best practices, including:
- Using secure and standard libraries
- Proper error handling
- Following PEP-8 guidelines
- Avoiding insecure coding patterns like SQL injection or hardcoded credentials

Please write a Python function for the following task: {task}
"""

In [103]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)


In [104]:
prompt_template = PromptTemplate(input_variables=["task"], template=task_prompt)
code_generation_chain = LLMChain(llm=llm, prompt=prompt_template)

In [105]:
from nemoguardrails import RailsConfig, LLMRails

In [106]:
path = "/content/guardrails_config.yaml"

In [45]:
!ls /content/


code_generation.colang	guardrails_config.yaml	sample_data


In [112]:
colang_content = """
# Define coding security flows
define user ask generate code
    "generate code"
    "write a function"
    "create a program"

define bot answer secure code
    "Here is your secure code:"

define flow secure_code
    user ask generate code
    bot answer secure code
    if match_code_pattern("AES.MODE_ECB")
        bot warn_user "ECB mode is insecure, please use CBC or GCM instead."
    if not match_code_pattern("try:\\n.*\\nexcept .*")
        bot warn_user "Code lacks proper error handling."
    bot offer_help
"""

# Define the YAML content specifying the LLM
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo
"""

In [108]:
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)


In [109]:
rails = LLMRails(config)


In [131]:
async def generate_and_validate_code(task_description):
    # Generate the code using Langchain asynchronously
    generated_code = await code_generation_chain.arun(task_description)  # Use async arun

    print("Generated Code:\n", generated_code)

    # Validate the code using NeMo Guardrails (synchronous run)
    result = rails.run(input_text=generated_code)

    # Check for violations or warnings in the code
    if result.violations:
        print("Code has violations:")
        for violation in result.violations:
            print(f"- {violation}")
    else:
        print("Code passed all guardrails and is ready for production!")

In [99]:
import asyncio

In [132]:
task_description = ["Write a function to encrypt a message using AES.",
                    "Write a function to generate a list of running devices"]

for description in task_description:
    await generate_and_validate_code(description)
#await generate_and_validate_code(task_description)  # Run async function


Generated Code:
 Here is a Python function that encrypts a message using AES encryption:

```python
import os
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_GCM)
    nonce = cipher.nonce

    ciphertext, tag = cipher.encrypt_and_digest(message.encode())

    return ciphertext, tag, nonce

# Example usage:
key = get_random_bytes(16)  # Generate a random 16-byte key
message = "Hello, World!"
ciphertext, tag, nonce = encrypt_message(message, key)

print("Ciphertext:", ciphertext)
print("Tag:", tag)
print("Nonce:", nonce)
```

This function uses the `Crypto` library for AES encryption, generates a random key using `get_random_bytes`, and encrypts the message using AES in GCM mode. It returns the ciphertext, tag, and nonce for decryption. 

Remember to securely store and manage the encryption key to ensure the security of the encrypted data.


AttributeError: 'LLMRails' object has no attribute 'run'